Annotation images → mask label for segmentation

In [2]:
import cv2
import numpy as np
import os

EDGE_DIR = "training_set/annos_edge"
MASK_DIR = "training_set/masks_filled"
os.makedirs(MASK_DIR, exist_ok=True)

for fname in os.listdir(EDGE_DIR):
    edge = cv2.imread(os.path.join(EDGE_DIR, fname), 0)
    _, edge = cv2.threshold(edge, 127, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(
        edge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE
    )
    if len(contours) == 0:
        continue

    cnt = max(contours, key=cv2.contourArea)
    filled = np.zeros_like(edge)
    cv2.drawContours(filled, [cnt], -1, 255, thickness=-1)

    cv2.imwrite(os.path.join(MASK_DIR, fname), filled)


Dataset load

In [ ]:
import torch
from torch.utils.data import Dataset
import cv2, pandas as pd, numpy as np

IMG_SIZE = 256

class FetalDataset(Dataset):
    def __init__(self, csv_path):
        self.df = pd.read_csv(csv_path)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img = cv2.imread(f"training_set/images/{row['filename']}", 0)
        mask = cv2.imread(f"training_set/masks_filled/{row['filename']}", 0)

        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE))

        img = img / 255.0
        mask = (mask > 0).astype(np.float32)

        img = np.stack([img]*3, axis=0)   # (3,H,W)
        mask = mask[None,:,:]             # (1,H,W)

        return (
            torch.tensor(img, dtype=torch.float32),
            torch.tensor(mask, dtype=torch.float32),
            row["pixel size(mm)"],
            row["head circumference (mm)"]
        )


U-Net pretrained (fine-tune)

In [24]:
import segmentation_models_pytorch as smp
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1
).to(DEVICE)
dice_loss = smp.losses.DiceLoss(mode="binary")
bce_loss = smp.losses.SoftBCEWithLogitsLoss()
def loss_fn(pred, target):
    return dice_loss(pred, target) + bce_loss(pred, target)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


Training

In [30]:
from torch.utils.data import DataLoader

dataset = FetalDataset("training_set/training_set_pixel_size_and_HC.csv")
loader = DataLoader(dataset, batch_size=8, shuffle=True)

model.train()
for epoch in range(20):
    print("Epoch loop entered", epoch)
    total_loss = 0
    for imgs, masks, _, _ in loader:
        imgs, masks = imgs.to(DEVICE), masks.to(DEVICE)

        preds = model(imgs)
        loss = loss_fn(preds, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}: loss={total_loss/len(loader):.4f}")


Epoch loop entered 0
Epoch 1: loss=0.2267
Epoch loop entered 1
Epoch 2: loss=0.1527
Epoch loop entered 2
Epoch 3: loss=0.1243
Epoch loop entered 3
Epoch 4: loss=0.0971
Epoch loop entered 4
Epoch 5: loss=0.0777
Epoch loop entered 5
Epoch 6: loss=0.0684
Epoch loop entered 6
Epoch 7: loss=0.0590
Epoch loop entered 7
Epoch 8: loss=0.0537
Epoch loop entered 8
Epoch 9: loss=0.0474
Epoch loop entered 9
Epoch 10: loss=0.0431
Epoch loop entered 10
Epoch 11: loss=0.0396
Epoch loop entered 11
Epoch 12: loss=0.0371
Epoch loop entered 12
Epoch 13: loss=0.0327
Epoch loop entered 13
Epoch 14: loss=0.0313
Epoch loop entered 14
Epoch 15: loss=0.0300
Epoch loop entered 15
Epoch 16: loss=0.0459
Epoch loop entered 16
Epoch 17: loss=0.0701
Epoch loop entered 17
Epoch 18: loss=0.0621
Epoch loop entered 18
Epoch 19: loss=0.0465
Epoch loop entered 19
Epoch 20: loss=0.0397


In [31]:
torch.save(model.state_dict(), "unet_hc.pth")

function to compute circumference of ellipse

In [ ]:
import math, cv2

def hc_from_pred(pred_mask, pixel_size):
    mask = (pred_mask > 0.5).astype(np.uint8)

    cnts,_ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if not cnts:
        return None

    cnt = max(cnts, key=cv2.contourArea)
    if len(cnt) < 5:
        return None

    (_, _), (maj, min_), _ = cv2.fitEllipse(cnt)
    a, b = maj/2, min_/2

    hc_px = math.pi * (3*(a+b) - math.sqrt((3*a+b)*(a+3*b)))
    return hc_px * pixel_size


evaluation

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for img, _, px, hc_gt in dataset:
        # read original image
        orig_img = cv2.imread(r"D:\USTH\MLmed\training_set\images", 0)
        orig_h, orig_w = orig_img.shape

        # preprocess
        img = cv2.resize(orig_img, (256, 256))
        img = img / 255.0
        img = np.stack([img]*3, axis=0)
        img = torch.tensor(img, dtype=torch.float32).unsqueeze(0)

        # predict
        pred = model(img.to(DEVICE))
        pred = torch.sigmoid(pred)[0, 0].cpu().numpy()

        # resize mask to original size
        pred = cv2.resize(
            pred,
            (orig_w, orig_h),
            interpolation=cv2.INTER_LINEAR
        )

        # compute HC (original pixel_size)
        hc = hc_from_pred(pred, px)
        if hc is not None:
            y_true.append(hc_gt)
            y_pred.append(hc)

y_true = np.array(y_true)
y_pred = np.array(y_pred)

print("MAE  (mm):", mean_absolute_error(y_true, y_pred))
print("MSE (mm²):", mean_squared_error(y_true, y_pred))
print("R²       :", r2_score(y_true, y_pred))


AttributeError: 'NoneType' object has no attribute 'shape'